# Evan, Icentia analysis

## importing necessary modules and initializing global variables

In [7]:
import numpy as np
from scipy.fft import rfft, rfftfreq
import matplotlib.pyplot as plt
import os
import scipy.signal as signal
from Tools.ReadPatientData import readSubject
from Tools.AnalysisTools import *
import neurokit2 as nk
import pandas as pd

dataFolder = "100data"      
fs = 250  

## Find missing files


In [3]:
import os
'''
# Define the directory where the files are located
directory = "ecg_rate_11k"  # Replace with your directory path

# Generate the expected filenames
expected_files = {f"{i:05}_ecg_rate.pkl.gz" for i in range(11000)}  # 00000 to 11000

actual_files = {files for files in os.listdir(directory)}

# Find the missing files
missing_files = expected_files - actual_files

if missing_files:
    print("Missing files:")
    for missing_file in sorted(missing_files):
        print(missing_file)
else:
    print("No missing files.")
'''

'\n# Define the directory where the files are located\ndirectory = "ecg_rate_11k"  # Replace with your directory path\n\n# Generate the expected filenames\nexpected_files = {f"{i:05}_ecg_rate.pkl.gz" for i in range(11000)}  # 00000 to 11000\n\nactual_files = {files for files in os.listdir(directory)}\n\n# Find the missing files\nmissing_files = expected_files - actual_files\n\nif missing_files:\n    print("Missing files:")\n    for missing_file in sorted(missing_files):\n        print(missing_file)\nelse:\n    print("No missing files.")\n'

## test, reading ecg_rate and ecg_clean files


In [ ]:
rate=pd.read_pickle('Test/00000_ecg_rate.pkl.gz', compression='gzip')
clean=pd.read_pickle('ecg_clean_filtered/00000_ecg_clean.pkl.gz', compression='gzip')
rows = clean["00000_subject"]
number_of_removal = rows[-1]
removed_sessions = rows[-2]
session = rows[:-2]

print(number_of_removal)
print(removed_sessions)
len(rows[0])


# Computing heart rate and fft


In [ ]:
import re
import plotly.graph_objects as go

def plot_heart_rate(folder, subject):
    '''
    Plots all session heart rate (both neurokit and annotaions genearted on top of each other to compare differences)

    parameters:
    - folder:  folder where it contains the file "*****_ecg_rate.pkl.gz"
    - subject: subject id
    '''
    rate=pd.read_pickle(f'{folder}/{subject:05d}_ecg_rate.pkl.gz', compression='gzip')
    rate_rows = rate[f"{subject:05d}_subject"]
    sessions = rate_rows.keys()
    session_numbers = [int(re.search(r'\d+$', s).group()) for s in sessions if re.search(r'\d+$', s)]

    for index in range(len(sessions)-2):
        ECGTotal, annotationTotal = readSubject(subject, dataFolder)
        session_id = session_numbers[index]
        ecg = ECGTotal[session_id]
        annotation = annotationTotal[session_id]
        peaks = findPeaks(annotation, len(ecg))
        peak_positions = np.where(peaks == 1)[0]
        
        ecg_rate_neurokit = rate_rows[index]
        ecg_rate_annotations= nk.signal_rate(peak_positions, sampling_rate=fs, desired_length=len(ecg))

        time = (np.arange(len(ecg)) / fs)[:len(ecg)//4]


        y1 = ecg_rate_annotations[:len(ecg)//4]
        y2 = ecg_rate_neurokit[:len(ecg)//4]

        plt.figure(figsize=(20,5))
        plt.plot(time, y2, label='ECG neurokit', alpha=0.5, linestyle='-' )
        plt.plot(time, y1, label='ECG annotations', alpha=0.5, linestyle='-')
       

        plt.xlabel('Time (s)')
        plt.ylabel('heart rate')
        plt.title(f'ecg_rate for subject_{subject:05d}, session_{session_id:02d}')
        plt.legend()
        plt.grid(True)
        plt.show()

plot_heart_rate('Test',8)

In [ ]:
def ecg_rate_superimposed_plot(folder, subject, session_id):
    '''
    Plot heart rate (generated by annotations and neruokit) of a single session from a single subject on top of each other (intearctive)
    parameters:
    - folder: folder where it contains the file "*****_ecg_rate.pkl.gz"
    - subject: subject id
    - session_id: session that you want to plot
    '''
    rate=pd.read_pickle(f'{folder}/{subject:05d}_ecg_rate.pkl.gz', compression='gzip')
    rate_rows = rate[f"{subject:05d}_subject"]
    sessions = rate_rows.keys()
    session_numbers = [int(re.search(r'\d+$', s).group()) for s in sessions if re.search(r'\d+$', s)]

    ECGTotal, annotationTotal = readSubject(subject, dataFolder)
    ecg = ECGTotal[session_id]
    annotation = annotationTotal[session_id]
    peaks = findPeaks(annotation, len(ecg))
    peak_positions = np.where(peaks == 1)[0]


    ecg_rate_neurokit = rate_rows[session_numbers.index(session_id)]
    ecg_rate_annotations= nk.signal_rate(peak_positions, sampling_rate=fs, desired_length=len(ecg))

    time = (np.arange(len(ecg)) / fs)
    y = ecg_rate_neurokit
    y1 = ecg_rate_annotations

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=time, y=y, mode='lines', name='neurokit'))
    fig.add_trace(go.Scatter(x=time, y=y1, mode='lines', name='annotations'))
    # Add the R-peaks as red markers

    # Update layout for better visualization
    fig.update_layout(
        title=f"Interactive heart rate plot, subject: {subject:05d}, session: {session_id:02d}",
        xaxis_title="Time (s)",
        yaxis_title="Amplitude",
        showlegend=True
    )
    # Show the plot
    fig.show()

ecg_rate_superimposed_plot('Test', 1,1)

In [ ]:
import plotly.graph_objects as go

def compare_peaks_in_ecg_raw(folder, subject_id, session_id):
    '''
    Outputs two interactive plot: 
    - ecg_raw with rpeaks labeled using neurokit
    - ecg_raw with rpeaks and noise labeled using annotations

    parameters:
    - folder: folder that contains the annotations
    - subject_id: the subject you want to compare the two methods with
    - session_id: the session you want to compare the two methods with

    '''
    ecg_raw_total, annotation_total = readSubject(subject_id,f"{folder}")
    ecg_raw = ecg_raw_total[session_id]

    l = len(ecg_raw)/fs
    x = (np.arange(len(ecg_raw)) / fs)
    y = ecg_raw

    ecg_clean = nk.ecg_clean(ecg_raw, sampling_rate=250, method="neurokit")
    _, rpeaks = nk.ecg_peaks(ecg_clean, sampling_rate=fs)
    rpeaks = (rpeaks['ECG_R_Peaks'])

    
    annotation = annotation_total[session_id]
    peaks = findPeaks(annotation, len(ecg_raw))
    noises = find_noise(annotation, len(ecg_raw))

    noise_positions = np.where(noises == 1)[0]
    peak_positions = np.where(peaks == 1)[0]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='Raw ECG Signal'))
    # Add the R-peaks as red markers
    fig.add_trace(go.Scatter(x=x[rpeaks], y=y[rpeaks], mode='markers', name='R-peaks', marker=dict(color='red')))
    # Update layout for better visualization
    fig.update_layout(
        title=f"(Neurokit) Interactive Raw ECG Signal with R-peaks, subject: {subject_id:05d}, session: {session_id:02d}",
        xaxis_title="Time (s)",
        yaxis_title="Amplitude",
        showlegend=True
    )
    # Show the plot
    fig.show()

    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x=x, y=y, mode='lines', name='Raw ECG Signal'))
    # Add the R-peaks as red markers
    fig1.add_trace(go.Scatter(x=x[peak_positions], y=y[peak_positions], mode='markers', name='R-peaks', marker=dict(color='red')))
    fig1.add_trace(go.Scatter(x=x[noise_positions], y=y[noise_positions], mode='markers', name='noises', marker=dict(color='yellow')))
    # Update layout for better visualization
    fig1.update_layout(
        title=f"(Annotations) Interactive Raw ECG Signal with R-peaks, subject: {subject_id:05d}, session: {session_id:02d}",
        xaxis_title="Time (s)",
        yaxis_title="Amplitude",
        showlegend=True
    )
    # Show the plot
    fig1.show()

compare_peaks_in_ecg_raw("100data",1,1
                         )


In [11]:
def process_ecg_rate(ecg_rate, detrend, filter_enabled, highcut, fs):
    '''
    pre-processing ecg_rate, detrend, filter, highcut
    
    parameters:
    - ecg_rate: the ecg rate beings processed
    - detrend: boolean values, determines to detrend or not
    - filter_enabled: boolean values, determines to filter or not
    - highcut: filter highcut
    - fs: sampling frequency
    '''
    if detrend:
        ecg_rate = signal.detrend(ecg_rate)

    if filter_enabled:
        ecg_rate = nk.signal_filter(
            ecg_rate,
            highcut=highcut,
            sampling_rate=fs,
        )
    return ecg_rate

def compute_fft(ecg_rate, fs):
    '''
    computes the fft given ecg_rate and fs
    '''
    N = len(ecg_rate)
    fft_values = np.abs(rfft(ecg_rate))
    freqs = rfftfreq(N, 1 / fs)
    
   
    window_size = 8
    smooth_fft = np.convolve(fft_values, np.ones(window_size) / window_size, mode='same')
    return freqs, smooth_fft

In [10]:
def sliding_window(data, window_time, overlap, noises, sampling_frequency=250):
    '''
    splits a session into segemnts given the desired time, overalp, and remove all the segments with noises

    parameters:
    - data: the session being segmented
    - window_time: time of the window in minutes
    - overlap: percent overlap you want (from 0 to 1)
    - noises: an array of 0 and 1 indicating the position of noise
    '''
    window_length = int(window_time * 60 * sampling_frequency)
    step_size = int(window_length * (1 - overlap))
    
    # Initialize dictionaries to hold windows with and without noise
    windows = {}
    removed_windows = {}
    segment = 0

    # Iterate over the data using the calculated step size and window length
    for start in range(0, len(data), step_size):
        window = data[start:start + window_length]
        noise_window = noises[start:start + window_length]
        
        if np.any(noise_window): 
            removed_windows[segment] = window
        else:
            windows[f'segment_{segment}'] = window
        
        segment += 1
    
    return windows, removed_windows

In [9]:
def calculate_differences(a_PSD,n_PSD):
    
    abs_differences = []
    for i in range (len(n_PSD)):
         _, a_fft = a_PSD[i]
         _, n_fft = n_PSD[i]
         a_fft = np.array(a_fft)
         n_fft = np.array(n_fft)

         differences = a_fft - n_fft
    
         abs_difference = np.sum(np.abs(differences))
         abs_differences.append(abs_difference)

    return(abs_differences)

In [15]:
import re
def save_psd(annotations_folder, neurokit_folder, subject, overlap,length, filter_threshold, plot=True):
    annotation_rates=pd.read_pickle(f'{annotations_folder}/{subject:05d}_ecg_rate.pkl.gz', compression='gzip')
    neurokit_rates=pd.read_pickle(f'{neurokit_folder}/{subject:05d}_ecg_rate.pkl.gz', compression='gzip')

    neurokit_rows = neurokit_rates[f"{subject:05d}_subject"][:-2]
    annotation_rows = annotation_rates[f"{subject:05d}_subject"][:-2]

    a_session_names = (annotation_rows.keys())
    n_session_names = (neurokit_rows.keys())
    common_session_names = list(set(a_session_names) & set(n_session_names))

    neurokit_rows = {key: value for key, value in neurokit_rows.items() if key in common_session_names}
    annotation_rows = {key: value for key, value in annotation_rows.items() if key in common_session_names}
   
    session_numbers = sorted([int(re.search(r'\d+$', s).group()) for s in common_session_names if re.search(r'\d+$', s)])

    neurokit_rates = list(neurokit_rows.values())
    annotation_rates = list(annotation_rows.values())

    _, annotations = readSubject(subject,"100data")
    number_of_sessions = len(common_session_names)

    session_results = {}
    for session in range(number_of_sessions):
        neurokit_rate = neurokit_rates[session]
        annotation_rate = annotation_rates[session]
        annotation = annotations[session_numbers[session]]

        noises = find_noise(annotation, len(neurokit_rate))
        a_windows, _ = sliding_window(annotation_rate,window_time=length, overlap=overlap, noises=noises)
        n_windows, n_removed_windows = sliding_window(neurokit_rate,window_time=length, overlap=overlap, noises=noises)

        a_segmented_rates = list(a_windows.values())
        segment_ids = list(a_windows.keys())

        n_segmented_rates = list(n_windows.values())

        results = {}
        for i in range(len(n_segmented_rates)):
            a_rate = a_segmented_rates[i]
            n_rate = n_segmented_rates[i]
            segment = segment_ids[i]
            if np.isnan(n_rate).any() or np.isnan(a_rate).any():
                continue
            try:
                a_rate_d = process_ecg_rate(a_rate, True, True, highcut = 0.35, fs=fs)
                n_rate_d = process_ecg_rate(n_rate, True, True, highcut = 0.35, fs=fs)

                a_freqs, a_fft = compute_fft(a_rate_d, fs)
                n_freqs, n_fft = compute_fft(n_rate_d, fs)

                a_fft = np.array(a_fft)
                n_fft = np.array(n_fft)
                differences = a_fft - n_fft
                abs_difference = np.sum(np.abs(differences))
                
                if abs_difference <= filter_threshold:
                    results[segment] = {
                        "freqs": a_freqs,
                        "a_fft": a_fft,
                        "n_fft": n_fft
                        }
                    
                    session_results[f'Session_{session_numbers[session]:02d}'] = results
        
                    if plot:
                        time = (np.arange(len(a_rate))/fs)/60
                        # Create a single figure with 4 subplots
                        plt.figure(figsize=(20, 10))

                        # Top-left subplot: (annotations) ECG Rate
                        plt.subplot(2, 2, 1)
                        plt.plot(time, a_rate, color='blue')
                        plt.xlabel('Time (minutes)')
                        plt.ylabel('ECG Rate')
                        plt.title(f'(annotations) ECG Rate (Subject_{subject:05d}_Session_{session_numbers[session]:02d}_Segment_{segment})')
                        plt.grid(True)

                        # Top-right subplot: (annotations) Power Spectral Density
                        plt.subplot(2, 2, 2)
                        plt.plot(a_freqs, a_fft, color='red')
                        plt.xlabel('Frequency (Hz)')
                        plt.ylabel('Magnitude')
                        plt.title(f'(annotations) Power Spectral Density (Subject_{subject:05d}_Session_{session_numbers[session]:02d}_Segment_{segment})')
                        plt.xlim(0, 0.35)
                        plt.grid(True)

                        # Bottom-left subplot: (neurokit) ECG Rate
                        plt.subplot(2, 2, 3)
                        plt.plot(time, n_rate, color='blue')
                        plt.xlabel('Time (minutes)')
                        plt.ylabel('ECG Rate')
                        plt.title(f'(neurokit) ECG Rate (Subject_{subject:05d}_Session_{session_numbers[session]:02d}_Segment_{segment})')
                        plt.grid(True)

                        # Bottom-right subplot: (neurokit) Power Spectral Density
                        plt.subplot(2, 2, 4)
                        plt.plot(n_freqs, n_fft, color='red')
                        plt.xlabel('Frequency (Hz)')
                        plt.ylabel('Magnitude')
                        plt.title(f'(neurokit) Power Spectral Density (Subject_{subject:05d}_Session_{session_numbers[session]:02d}_Segment_{segment})')
                        plt.xlim(0, 0.35)
                        plt.grid(True)
                        # Adjust layout to prevent overlapping
                        plt.tight_layout()

                        # Show the figure
                        plt.show()
                        # Print the absolute differences
                        print(f"total absolute differnces: {abs_difference}")
                    
                session_results[f"Session_{session_numbers[session]:02d}"] = results
        
            except:
                print(f'error proceesing (Subject_{subject:05d}_Session_{session_numbers[session]:02d}_Segment_{segment})')
            
                
        if plot:
            plt.figure(figsize=(16, 5))
            whole_time = (np.arange(len(neurokit_rate)) /fs)/ 60
            plt.plot(whole_time, neurokit_rate, color='blue', label='ECG_rate Data')

            for removed in n_removed_windows.keys():
                start = removed * (length * (1 - overlap))
                end = start + length
                plt.axvspan(start, end, color='red', alpha=0.3)
            
            plt.xlabel('Time (minutes)')
            plt.ylabel('ECG rate Data')
            plt.title(f'Entire Heart Rate Data (Subject_{subject:05d}, Session {session_numbers[session]}) with Removed Segments Highlighted')
            plt.legend()
            plt.grid(True)
            plt.show()
        
    
            
    results_df = pd.DataFrame(session_results).T
    return results_df#session_results

results = save_psd(annotations_folder='ECG_rate_annotations', neurokit_folder="Test", subject=0,overlap=0, length=5, filter_threshold=2e5, plot=False)

In [16]:
results = results.T

# Test FOOOF

In [ ]:
# Import required code for visualizing example models
from fooof import FOOOF
from fooof.sim.gen import gen_power_spectrum
from fooof.sim.utils import set_random_seed
from fooof.plts.spectra import plot_spectra
from fooof.plts.annotate import plot_annotated_model

In [ ]:
# Initialize power spectrum model objects and fit the power spectra
freq_range = [0, 0.5]
fm = FOOOF(peak_width_limits=[0.02, 0.06], max_n_peaks=5, min_peak_height=0.2,
           peak_threshold=2.0, aperiodic_mode='knee')
# Model the power spectrum with FOOOF, and print out a report
fm.report(freqs, a_fft, freq_range)

In [ ]:
fm = FOOOF(peak_width_limits=[0.02, 0.06], max_n_peaks=6, min_peak_height=0.19,
           peak_threshold=2.0, aperiodic_mode='knee')
# Model the power spectrum with FOOOF, and print out a report


for session in results.keys():
    segments = results[session]
    for segment_id in segments.keys():
        segment = segments[segment_id]
        if segment is None or (isinstance(segment, float) and np.isnan(segment)):
    
            continue
        a_fft = segment['a_fft']
        freqs = segment['freqs']
        freqs = freqs[(freqs >= 0.01) & (freqs <= 0.5)]
        a_fft = a_fft[:len(freqs)]
        fm.report(freqs, a_fft, freq_range)
        plt.figure(figsize=(10,5))
        plt.plot(freqs,a_fft, color='blue')
        plt.xlabel('freqs')
        plt.ylabel('power')
        plt.title('psd')
        plt.grid(True)

        

## counts the availables segments and how many contains mayer waves

In [ ]:
from fooof import FOOOF
import numpy as np
import pandas as pd

def mayer_wave_counts(PSDs, show=True):
    """
    Analyzes PSDs to count the number of valid (non-NaN) PSDs and those containing Mayer waves (0.05 - 0.15 Hz).
    Also returns the corresponding session names and segment names for PSDs containing Mayer waves.

    Parameters:
    - PSDs: DataFrame where each entry is a PSD dictionary (with 'freqs' and 'a_fft') or NaN.
    - show: If True, prints summary statistics.

    Returns:
    - total_valid_psds: Number of non-NaN PSDs analyzed.
    - mayer_wave_count: Number of PSDs containing Mayer waves.
    - mayer_wave_segments: List of tuples (session_name, segment_name) where Mayer waves were detected.
    """

    fm = FOOOF(peak_width_limits=[0.02, 0.06], max_n_peaks=5, min_peak_height=0.2,
               peak_threshold=2.0, aperiodic_mode='knee')

    mayer_wave_range = (0.05, 0.15)
    total_valid_psds = 0
    mayer_wave_count = 0
    mayer_wave_segments = []  # Store session and segment names

    for session in PSDs.columns:
        for segment in PSDs.index:
            entry = PSDs.at[segment, session]

            if isinstance(entry, dict) and 'freqs' in entry and 'a_fft' in entry:
                freqs = np.array(entry['freqs'])
                power_spectrum = np.array(entry['a_fft'])

                valid_idx = freqs > 0  # Remove 0 Hz
                freqs = freqs[valid_idx]
                power_spectrum = power_spectrum[valid_idx]

                if len(freqs) > 0 and len(power_spectrum) > 0:
                    fm.fit(freqs, power_spectrum, [0, 0.35])
                    total_valid_psds += 1

                    for peak in fm.get_params('peak_params'):
                        peak = np.array(peak)
                        if np.isnan(peak).any():
                            continue
                        peak_freq = peak[0]
                        if mayer_wave_range[0] <= peak_freq <= mayer_wave_range[1]:
                            mayer_wave_count += 1
                            mayer_wave_segments.append((session, segment))
                            break  # Count only once per PSD

    if show:                   
        print(f"Total non-NaN PSDs: {total_valid_psds}")
        print(f"Number of PSDs containing Mayer waves: {mayer_wave_count}")
        print("Sessions and Segments with Mayer waves:")
        for session, segment in mayer_wave_segments:
            print(f"Session: {session}, Segment: {segment}")

    return total_valid_psds, mayer_wave_count, mayer_wave_segments



mayer_wave_counts(results, show = True)

# Test for cutoff threshold

In [15]:

diff = calculate_differences(a_PSD, n_PSD)


NameError: name 'calculate_differences' is not defined

In [ ]:
a_PSDs, n_PSDs = [],[]
for i in range(100):
    a_PSD, n_PSD = save_psd(annotations_folder='ECG_rate_annotations', neurokit_folder="Test", subject=i,overlap=0, length=5, plot=False)
    a_PSDs.append(a_PSD)
    n_PSDs.append(n_PSD)

    

In [ ]:
import matplotlib.pyplot as plt
all_diffs = []
fig = plt.figure(figsize=(15, 4))
fig, ax = plt.subplots(figsize=(15, 4))
plt.xlabel("Difference Value")
plt.ylabel("Frequency")

#ax.set_xlim(0,1e7)
ax.set_title("absolute differences histograms for 100 subjects")
for i in range(len(a_PSDs)):
    a_PSD = a_PSDs[i]
    n_PSD = n_PSDs[i]
    diff = calculate_differences(a_PSD,n_PSD)
    ax.hist(diff, alpha=0.5, edgecolor='black',bins=30)
    all_diffs.append(diff)




In [ ]:
flattened_all_diff = [item for sublist in all_diffs for item in sublist]

counts, bins, _ = plt.hist(flattened_all_diff, bins = 100, alpha = 0.8, edgecolor = "black")
plt.title("Histogram_100_subjects")
plt.xlabel("Difference Value")
plt.ylabel("Frequency")

peak_bin_index = np.argmax(counts)  
peak_value = (bins[peak_bin_index]) 

In [ ]:
neurokit_rates=pd.read_pickle(f'Test/00080_ecg_rate.pkl.gz', compression='gzip')
annotation_rates=pd.read_pickle('ECG_rate_annotations/00080_ecg_rate.pkl.gz', compression='gzip')

neurokit_rates
annotation_rates

In [ ]:
def filter_PSD(a_PSDs,n_PSDs,threshold):
    filtered_a_PSDs = []
    filtered_n_PSDs = []

    for i in range(len(a_PSDs)):
        a_PSD = a_PSDs[i]
        n_PSD = n_PSDs[i]
        differences = calculate_differences(a_PSD,n_PSD)

        filtered_a_segments = []
        filtered_n_segments = []

        for j in range(len(differences)):
            if differences[j] <= threshold: 
                filtered_a_segments.append(a_PSD[j])  
                filtered_n_segments.append(n_PSD[j]) 

        if filtered_a_segments and filtered_n_segments:
            filtered_a_PSDs.append(filtered_a_segments)
            filtered_n_PSDs.append(filtered_n_segments)

    return filtered_a_PSDs, filtered_n_PSDs

a,b = filter_PSD(a_PSDs,n_PSDs, 5e5)


In [ ]:
c = []
for i in range(len(a)):
    a1 = a[i]
    b1 = b[i]
    diff = calculate_differences(a1,b1)
    c.append(diff)


In [ ]:
d = [item for sublist in c for item in sublist]


fig = plt.figure(figsize=(6, 4))
plt.xlim(0,1e6)

counts, bins, _ = plt.hist(d, bins = 3, alpha = 0.8, edgecolor = "black")
plt.title("Histogram_100_subjects")
plt.xlabel("Difference Value")
plt.ylabel("Frequency")

NameError: name 'c' is not defined

In [ ]:
len(d)/len(flattened_all_diff)

NameError: name 'd' is not defined